In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupShuffleSplit 
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
buf = np.load("../valid_files.npy", allow_pickle=True).item()
health = buf["health"]
mdd = buf["mdd"]

In [3]:
df = pd.read_csv('ml_prepared_data/env.csv')
columns = df.columns
columns[1:-2]

Index(['env_beta_Fp1_Fp2', 'env_beta_Fp1_Fpz', 'env_beta_Fp1_F3',
       'env_beta_Fp1_F4', 'env_beta_Fp1_Fz', 'env_beta_Fp1_C3',
       'env_beta_Fp1_C4', 'env_beta_Fp1_Cz', 'env_beta_Fp1_P3',
       'env_beta_Fp1_P4',
       ...
       'env_beta_F8_T3', 'env_beta_F8_T4', 'env_beta_F8_T5', 'env_beta_F8_T6',
       'env_beta_T3_T4', 'env_beta_T3_T5', 'env_beta_T3_T6', 'env_beta_T4_T5',
       'env_beta_T4_T6', 'env_beta_T5_T6'],
      dtype='object', length=210)

In [4]:
df = pd.read_csv('ml_prepared_data/entropy_envelopes_band_power_features.csv')
df["name"] = df["name"].apply(lambda x: x + ".npy")
df = df[df["name"].isin(mdd) | df["name"].isin(health)]
df.drop(list(columns[1:-2]), axis=1, inplace=True)

In [5]:
df["patient"] = df["name"].apply(lambda x: x.split("_")[1])
del df["name"]
df.shape
df.drop(["Unnamed: 0"],axis=1, inplace=True)
df.head()

,entropy_bands_alpha_Fp1,entropy_bands_beta_Fp1,entropy_bands_gamma_Fp1,entropy_bands_theta_Fp1,entropy_bands_alpha_Fp2,entropy_bands_beta_Fp2,entropy_bands_gamma_Fp2,entropy_bands_theta_Fp2,entropy_bands_alpha_Fpz,entropy_bands_beta_Fpz,...,feats_bands_alpha_T5,feats_bands_beta_T5,feats_bands_gamma_T5,feats_bands_theta_T5,feats_bands_alpha_T6,feats_bands_beta_T6,feats_bands_gamma_T6,feats_bands_theta_T6,target,patient
0,-13.728290,-13.624056,-15.834408,-13.225958,-13.582560,-13.593790,-15.657767,-13.125128,-13.571858,-13.568165,...,0.117739,0.151308,0.002834,0.138549,0.093583,0.078777,0.001882,0.122073,1,F117
1,-12.729169,-13.587557,-15.786211,-12.958005,-12.980654,-13.779130,-16.216976,-12.792485,-12.892763,-13.563334,...,0.223667,0.165292,0.004435,0.156241,0.512208,0.094983,0.001061,0.072389,1,M163
2,-13.163862,-13.141093,-15.270868,-13.024770,-12.755880,-12.781630,-15.052215,-12.727004,-12.919386,-13.201586,...,0.126807,0.166286,0.006457,0.255623,0.123857,0.177574,0.005008,0.264014,1,F130
3,-13.232000,-13.632697,-15.843904,-12.818745,-13.363398,-13.719045,-16.058201,-12.897723,-13.235419,-13.576650,...,0.110412,0.118039,0.002796,0.157801,0.131513,0.108960,0.001644,0.107869,1,M163
4,-13.427979,-13.784442,-15.839615,-13.062695,-13.314286,-13.630312,-15.696986,-12.802822,-13.416008,-13.680343,...,0.099580,0.127066,0.001272,0.251802,0.098100,0.132150,0.001493,0.366939,1,F117


In [6]:
splitter = GroupShuffleSplit(test_size=.3, n_splits=2, random_state = 7)
split = splitter.split(df, groups=df['patient'])
train_inds, test_inds = next(split)

train = df.iloc[train_inds]
test = df.iloc[test_inds]
test["patient"].unique()
test.shape

(869, 170)

In [7]:
y_train = train["target"].values
del train["target"]
train_group = train["patient"].values
del train["patient"]

y_test = test["target"].values
del test["target"]
test_group = test["patient"].values
del test["patient"]

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

splitter_cv = GroupShuffleSplit(test_size=.2, n_splits=5, random_state = 42)
split_cv = splitter.split(train, y_train, groups=train_group)



In [9]:
from sklearn.linear_model import LogisticRegression
R = LogisticRegression(max_iter = 5000)

grid = {
    'C':np.linspace(0.001, 0.01, 100)
}

gs = GridSearchCV(R, param_grid=grid, cv=split_cv, scoring="accuracy")
gs.fit(train, y_train)


y_gs = gs.predict(train)
print(accuracy_score(y_train, y_gs))
y_gs = gs.predict(test)
print(accuracy_score(y_test, y_gs))
print(gs.best_estimator_)

0.7641325536062378
0.667433831990794
LogisticRegression(C=0.009272727272727273, max_iter=5000)
